In [1]:
import pandas as pd

test_data_split = 230  #this will be the number of weeks in the test data set, remainder of data in train data set
#Calculate the split date to use
split_date = datetime.now() - timedelta(weeks=test_data_split) 
print('Split Date: {0}'.format(split_date))

OBS_DATE                   datetime64[ns]
RAIL_FRT_CARLOADS_D11               int64
RAIL_FRT_INTERMODAL_D11             int64
WATERBORNE_D11                    float64
TRUCK_D11                         float64
AIR_RTMFM_D11                       int64
TSI                               float64
dtype: object

In [ ]:
#Read in the transportation data, monthly seasonally adjusted
tsi_data = pd.read_excel(".\data\Input_SeasonalData_TSI.xlsx", header=2)

#look at the data types that were inferred by Pandas during import.
tsi_data.dtypes

In [2]:
#Return a listing of the data
tsi_data.head()

,OBS_DATE,RAIL_FRT_CARLOADS_D11,RAIL_FRT_INTERMODAL_D11,WATERBORNE_D11,TRUCK_D11,AIR_RTMFM_D11,TSI
0,2000-01-01,1422442,764756,55.4,80.3,2466950,105.3
1,2000-02-01,1425882,767958,48.6,79.8,2521852,104.4
2,2000-03-01,1411458,763858,52.5,74.1,2489787,99.2
3,2000-04-01,1400311,764144,50.8,72.8,2557332,98.1
4,2000-05-01,1405169,763843,52.5,73.0,2527821,98.6


In [3]:
#Read in the real gdp growth rates,seasonally adjusted, quarterly data with quarterly growth rates
gdp_data = pd.read_excel(".\data\Input_GDP_st_louis_fed_quarterly_change.xlsx")

#look at the data types that were inferred by Pandas during import.
gdp_data.dtypes

observation_date       datetime64[ns]
Real_gdp_qtr_growth           float64
dtype: object

In [4]:
#Disply a listing of the data
gdp_data.head()

,observation_date,Real_gdp_qtr_growth
0,2000-01-01,1.5
1,2000-04-01,7.5
2,2000-07-01,0.5
3,2000-10-01,2.5
4,2001-01-01,-1.1


In [5]:
#Merge the 2 datasets together based on the date. 
merged_data = pd.merge(tsi_data, gdp_data, how='left', left_on='OBS_DATE', right_on='observation_date')

merged_data.head()

,OBS_DATE,RAIL_FRT_CARLOADS_D11,RAIL_FRT_INTERMODAL_D11,WATERBORNE_D11,TRUCK_D11,AIR_RTMFM_D11,TSI,observation_date,Real_gdp_qtr_growth
0,2000-01-01,1422442,764756,55.4,80.3,2466950,105.3,2000-01-01,1.5
1,2000-02-01,1425882,767958,48.6,79.8,2521852,104.4,NaT,NaN
2,2000-03-01,1411458,763858,52.5,74.1,2489787,99.2,NaT,NaN
3,2000-04-01,1400311,764144,50.8,72.8,2557332,98.1,2000-04-01,7.5
4,2000-05-01,1405169,763843,52.5,73.0,2527821,98.6,NaT,NaN


In [6]:
#We now have 2 date columns.  Drop the date column from the gdp_data dataframe.
merged_data = merged_data.drop(['observation_date'], axis=1) #pandas can drop rows or columns, axis=1 indicates columns

In [7]:
#This now shows the dataframe without the duplicate date column observation_date.
merged_data.head()

,OBS_DATE,RAIL_FRT_CARLOADS_D11,RAIL_FRT_INTERMODAL_D11,WATERBORNE_D11,TRUCK_D11,AIR_RTMFM_D11,TSI,Real_gdp_qtr_growth
0,2000-01-01,1422442,764756,55.4,80.3,2466950,105.3,1.5
1,2000-02-01,1425882,767958,48.6,79.8,2521852,104.4,NaN
2,2000-03-01,1411458,763858,52.5,74.1,2489787,99.2,NaN
3,2000-04-01,1400311,764144,50.8,72.8,2557332,98.1,7.5
4,2000-05-01,1405169,763843,52.5,73.0,2527821,98.6,NaN


In [8]:
#Look at the counts for the data values that we have for each column.  We see the gdp data with a smaller number since
#it is quarterly and the other data is monthly.
merged_data.count()

OBS_DATE                   241
RAIL_FRT_CARLOADS_D11      241
RAIL_FRT_INTERMODAL_D11    241
WATERBORNE_D11             241
TRUCK_D11                  241
AIR_RTMFM_D11              241
TSI                        241
Real_gdp_qtr_growth         80
dtype: int64

In [9]:
#Forward fill the gdp data so that a quarterly gdp value will be used for 3 rows, with each row being a month.
#limit it only fill in 2 consective missing values.
merged_data['Real_gdp_qtr_growth'] = merged_data['Real_gdp_qtr_growth'].fillna(method='ffill', limit=2)

merged_data.head()

,OBS_DATE,RAIL_FRT_CARLOADS_D11,RAIL_FRT_INTERMODAL_D11,WATERBORNE_D11,TRUCK_D11,AIR_RTMFM_D11,TSI,Real_gdp_qtr_growth
0,2000-01-01,1422442,764756,55.4,80.3,2466950,105.3,1.5
1,2000-02-01,1425882,767958,48.6,79.8,2521852,104.4,1.5
2,2000-03-01,1411458,763858,52.5,74.1,2489787,99.2,1.5
3,2000-04-01,1400311,764144,50.8,72.8,2557332,98.1,7.5
4,2000-05-01,1405169,763843,52.5,73.0,2527821,98.6,7.5


In [10]:
#Look to see if we still have missing gdp values as that data gets released later than the transportation data.
#This will allow us to inspect any bad rows.
merged_data.loc[merged_data['Real_gdp_qtr_growth'].isnull()]

,OBS_DATE,RAIL_FRT_CARLOADS_D11,RAIL_FRT_INTERMODAL_D11,WATERBORNE_D11,TRUCK_D11,AIR_RTMFM_D11,TSI,Real_gdp_qtr_growth
240,2020-01-01,1054432,1109662,50.4,116.6,3617962,136.9,NaN


In [11]:
#drop rows where we don't have a gdp value
merged_data = merged_data.dropna(subset=['Real_gdp_qtr_growth'])

In [12]:
#Verify that all rows have gdp values 
merged_data.loc[merged_data['Real_gdp_qtr_growth'].isnull()]

,OBS_DATE,RAIL_FRT_CARLOADS_D11,RAIL_FRT_INTERMODAL_D11,WATERBORNE_D11,TRUCK_D11,AIR_RTMFM_D11,TSI,Real_gdp_qtr_growth


In [13]:
#We should now have clean data.
merged_data.count()

OBS_DATE                   240
RAIL_FRT_CARLOADS_D11      240
RAIL_FRT_INTERMODAL_D11    240
WATERBORNE_D11             240
TRUCK_D11                  240
AIR_RTMFM_D11              240
TSI                        240
Real_gdp_qtr_growth        240
dtype: int64

In [14]:
#Create a boolean column to indicate if gdp growth was positive or negative.
merged_data['gdp_is_increasing'] = merged_data.apply(lambda x: True if x['Real_gdp_qtr_growth'] > 0.0 else False, axis=1)

In [15]:
merged_data.count()

OBS_DATE                   240
RAIL_FRT_CARLOADS_D11      240
RAIL_FRT_INTERMODAL_D11    240
WATERBORNE_D11             240
TRUCK_D11                  240
AIR_RTMFM_D11              240
TSI                        240
Real_gdp_qtr_growth        240
gdp_is_increasing          240
dtype: int64

In [16]:
#look at the new column
merged_data.head()

,OBS_DATE,RAIL_FRT_CARLOADS_D11,RAIL_FRT_INTERMODAL_D11,WATERBORNE_D11,TRUCK_D11,AIR_RTMFM_D11,TSI,Real_gdp_qtr_growth,gdp_is_increasing
0,2000-01-01,1422442,764756,55.4,80.3,2466950,105.3,1.5,True
1,2000-02-01,1425882,767958,48.6,79.8,2521852,104.4,1.5,True
2,2000-03-01,1411458,763858,52.5,74.1,2489787,99.2,1.5,True
3,2000-04-01,1400311,764144,50.8,72.8,2557332,98.1,7.5,True
4,2000-05-01,1405169,763843,52.5,73.0,2527821,98.6,7.5,True


In [17]:
#Look at rows where gdp growth was negative
merged_data.loc[merged_data['Real_gdp_qtr_growth'] <= 0.0]

,OBS_DATE,RAIL_FRT_CARLOADS_D11,RAIL_FRT_INTERMODAL_D11,WATERBORNE_D11,TRUCK_D11,AIR_RTMFM_D11,TSI,Real_gdp_qtr_growth,gdp_is_increasing
12,2001-01-01,1398488,759514,46.0,74.8,2553409,99.5,-1.1,False
13,2001-02-01,1399176,743794,47.6,75.3,2547226,99.9,-1.1,False
14,2001-03-01,1416765,754615,47.0,74.2,2479860,99.1,-1.1,False
18,2001-07-01,1353431,731670,47.6,74.1,2246636,97.6,-1.7,False
19,2001-08-01,1374005,745111,49.3,75.2,2318149,99.2,-1.7,False
20,2001-09-01,1376994,754272,49.0,74.4,2047594,98.1,-1.7,False
96,2008-01-01,1416854,985481,45.1,87.5,3366557,113.4,-2.3,False
97,2008-02-01,1423289,998459,43.1,85.8,3289484,111.5,-2.3,False
98,2008-03-01,1402813,971721,36.7,85.8,3249574,109.8,-2.3,False
102,2008-07-01,1427847,986893,42.5,86.2,3126320,111.1,-2.1,False


In [18]:
merged_data.head()

,OBS_DATE,RAIL_FRT_CARLOADS_D11,RAIL_FRT_INTERMODAL_D11,WATERBORNE_D11,TRUCK_D11,AIR_RTMFM_D11,TSI,Real_gdp_qtr_growth,gdp_is_increasing
0,2000-01-01,1422442,764756,55.4,80.3,2466950,105.3,1.5,True
1,2000-02-01,1425882,767958,48.6,79.8,2521852,104.4,1.5,True
2,2000-03-01,1411458,763858,52.5,74.1,2489787,99.2,1.5,True
3,2000-04-01,1400311,764144,50.8,72.8,2557332,98.1,7.5,True
4,2000-05-01,1405169,763843,52.5,73.0,2527821,98.6,7.5,True


In [19]:
merged_data['RAIL_FRT_PCT'] = merged_data['RAIL_FRT_CARLOADS_D11'].pct_change()
merged_data['RAIL_INTERMOD_PCT'] = merged_data['RAIL_FRT_INTERMODAL_D11'].pct_change()
merged_data['WATERBORNE_PCT'] = merged_data['WATERBORNE_D11'].pct_change()
merged_data['TRUCK_PCT'] = merged_data['TRUCK_D11'].pct_change()
merged_data['AIR_RTMFM_PCT'] = merged_data['AIR_RTMFM_D11'].pct_change()
merged_data['TSI_PCT'] = merged_data['TSI'].pct_change()

In [20]:
from datetime import datetime, timedelta

#create x_train and y_train dataframes
x_train = merged_data.loc[merged_data['OBS_DATE'] < split_date, 'RAIL_FRT_PCT':'TSI_PCT']
x_train.drop([0], inplace=True) #drop first row as it will have NA for pct_change values
y_train = merged_data.loc[merged_data['OBS_DATE'] < split_date, 'gdp_is_increasing']
y_train.drop([0], inplace=True) #drop first row as it will have NA for pct_change values

print('x_train row count: {0}'.format(len(x_train.index)))
print('y_train row count: {0}'.format(len(y_train.index)))

x_test = merged_data.loc[merged_data['OBS_DATE'] >= split_date, 'RAIL_FRT_PCT':'TSI_PCT']
y_test = merged_data.loc[merged_data['OBS_DATE'] >= split_date, 'gdp_is_increasing']

print('x_test row count: {0}'.format(len(x_test.index)))
print('y_test row count: {0}'.format(len(y_test.index)))

Split Date: 2015-11-23 22:24:54.504225
x_train row count: 190
y_train row count: 190
x_test row count: 49
y_test row count: 49


In [21]:
#x_train

In [22]:
from sklearn import preprocessing

mm_scaler = preprocessing.MinMaxScaler()
x_train_minmax = mm_scaler.fit_transform(x_train)

x_test_minmax = mm_scaler.transform(x_test)

In [23]:
#x_train_minmax

In [24]:
#x_test_minmax

In [25]:
y_train.head()

1    True
2    True
3    True
4    True
5    True
Name: gdp_is_increasing, dtype: bool

In [26]:
from sklearn.linear_model import LogisticRegression
import numpy as np

#load the model
clf = LogisticRegression(solver='liblinear')
#fit the model
clf.fit(x_train_minmax, y_train)

#evaluate the model by using a test set
y_hat = clf.predict(x_test_minmax)
#print the accuracy
print(np.average(y_hat == y_test))

print('Done')

1.0
Done


In [27]:
y_hat

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [28]:
x_test

,RAIL_FRT_PCT,RAIL_INTERMOD_PCT,WATERBORNE_PCT,TRUCK_PCT,AIR_RTMFM_PCT,TSI_PCT
191,-0.015480,-0.022111,-0.050104,0.014042,0.011569,-0.004132
192,-0.021953,0.049241,-0.054945,0.015826,0.000809,0.016598
193,-0.003973,0.001840,0.072093,-0.005842,-0.032887,-0.006531
194,-0.024305,-0.039484,-0.034707,-0.002938,0.024997,-0.009860
195,-0.012144,-0.009430,0.020225,0.013752,0.009591,0.020747
196,0.007258,0.012907,-0.002203,-0.010659,-0.014564,-0.006504
197,0.028067,-0.007228,-0.026490,0.001959,0.024767,0.007365
198,0.027701,-0.002657,0.070295,0.026393,-0.001024,0.020309
199,-0.001436,0.009055,-0.019068,-0.034286,0.013866,-0.016720
200,0.016810,0.000391,0.015119,-0.012821,0.015345,-0.010526


In [29]:
y_test

191    True
192    True
193    True
194    True
195    True
196    True
197    True
198    True
199    True
200    True
201    True
202    True
203    True
204    True
205    True
206    True
207    True
208    True
209    True
210    True
211    True
212    True
213    True
214    True
215    True
216    True
217    True
218    True
219    True
220    True
221    True
222    True
223    True
224    True
225    True
226    True
227    True
228    True
229    True
230    True
231    True
232    True
233    True
234    True
235    True
236    True
237    True
238    True
239    True
Name: gdp_is_increasing, dtype: bool

In [30]:
merged_data

,OBS_DATE,RAIL_FRT_CARLOADS_D11,RAIL_FRT_INTERMODAL_D11,WATERBORNE_D11,TRUCK_D11,AIR_RTMFM_D11,TSI,Real_gdp_qtr_growth,gdp_is_increasing,RAIL_FRT_PCT,RAIL_INTERMOD_PCT,WATERBORNE_PCT,TRUCK_PCT,AIR_RTMFM_PCT,TSI_PCT
0,2000-01-01,1422442,764756,55.4,80.3,2466950,105.3,1.5,True,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-02-01,1425882,767958,48.6,79.8,2521852,104.4,1.5,True,0.002418,0.004187,-0.122744,-0.006227,0.022255,-0.008547
2,2000-03-01,1411458,763858,52.5,74.1,2489787,99.2,1.5,True,-0.010116,-0.005339,0.080247,-0.071429,-0.012715,-0.049808
3,2000-04-01,1400311,764144,50.8,72.8,2557332,98.1,7.5,True,-0.007898,0.000374,-0.032381,-0.017544,0.027129,-0.011089
4,2000-05-01,1405169,763843,52.5,73.0,2527821,98.6,7.5,True,0.003469,-0.000394,0.033465,0.002747,-0.011540,0.005097
5,2000-06-01,1394702,755760,48.5,74.9,2519527,99.6,7.5,True,-0.007449,-0.010582,-0.076190,0.026027,-0.003281,0.010142
6,2000-07-01,1400222,758084,49.6,72.7,2502082,97.5,0.5,True,0.003958,0.003075,0.022680,-0.029372,-0.006924,-0.021084
7,2000-08-01,1401724,755196,48.3,74.7,2579427,99.6,0.5,True,0.001073,-0.003810,-0.026210,0.027510,0.030912,0.021538
8,2000-09-01,1417025,766163,48.5,74.1,2759673,99.7,0.5,True,0.010916,0.014522,0.004141,-0.008032,0.069878,0.001004
9,2000-10-01,1395338,781620,47.8,74.2,2649911,99.5,2.5,True,-0.015305,0.020175,-0.014433,0.001350,-0.039774,-0.002006
